# 1. llama.cpp

创始人：Georgi Gerganov。

llama.cpp是一个github上的开源项目。
它的主要目标是在各种硬件上（本地和云端）以最少的设置实现大语言模型推理，并达到最先进的性能。

- Ollama的底层就是llama.cpp这个项目。Ollama更侧重用户交互，而llama.cpp更侧重推理的性能。
    - ollama安装更方便。
    - 提供了 [model library在线仓库](https://ollama.com/library) 。
- 底层语言：C/C++。
- 模型文件格式要求为GGUF（文件后缀为.gguf）。（huggingface上的模型通常是 PyTorch(后缀为.pt/.pth) 或 SafeTensors 格式(后缀为.safetensors)）
    - GGUF (GGML Universal File), [来源](https://pypi.org/project/gguf/) 
        -  ~~GGUF 是 "GPT-Generated Unified Format" 的缩写，中文名称为：“GPT 生成的统一格式”。~~
    -  GGUF 是一种专为LLM 模型设计的二进制文件格式，由开源项目llama.cpp 的创始人Georgi Gerganov 提出。
    -  GGUF格式侧重于提升模型加载和推理过程中的性能和效率。


## 1.1 llama.cpp的安装

- macos
    ```shell
        brew install llama.cpp
    ```
    
- docker镜像，[地址](https://github.com/ggml-org/llama.cpp/blob/master/docs/docker.md)

    - ghcr.io/ggml-org/llama.cpp:full, 包含了llama.cpp的所有功能，大小2G
    - ghcr.io/ggml-org/llama.cpp:light, 只包含主可执行文件,大小为94M
    - ghcr.io/ggml-org/llama.cpp:server，只包含server的可执行文件,大小96M
    - ......
  
- 预编译好的文件，[地址](https://github.com/ggml-org/llama.cpp/releases) ,windows等。

## 1.2 命令行和转换工具

- llama-cli: 一个用于访问和试验大部分 llama.cpp 功能的命令行界面工具。
- **llama-server**: 一个轻量级的，OpenAI API兼容的用于服务大语言模型的 HTTP 服务器。
- llama-bench: 对各种参数下的推理性能进行基准测试。
- llama-run: 在命令行中运行LLM，和ollama run 一样。
- llama-sample: 根据参数运行一次，运行结束后退出。（对开发人员比较有用）

格式转换工具（将其他模型文件转换成gguf格式）

- convert_hf_to_gguf.py
    -  将 Hugging Face 格式的模型权重（通常是 PyTorch 或 SafeTensors 格式）转换为 GGUF 格式。
    
- convert_hf_to_gguf_update.py
    - 用于更新convert_hf_to_gguf.py工具中的get_vocab_base_pre()函数
    - get_vocab_base_pre(): 根据分词器获取模型类型的一个函数。
    - 一般不需要执行这个工具，除非遇到下面的错误：
        - NotImplementedError("BPE pre-tokenizer was not recognized - update get_vocab_base_pre()")

- convert_llama_ggml_to_gguf.py
    - 将旧版的 GGML 格式模型文件转换为新版的 GGUF 格式。
    - GGML文件格式，它是gguf的前身，也是llama.cpp的作者提出。随着llama.cpp项目的演进和发展，这种格式后续会慢慢被替换掉。
    - ~~GGML，General GPU Management Layer，通用GPU管理层。~~
    - ggml 是 Georgi Gerganov Machine Learning 的缩写，其命名来源于项目的核心开发者 Georgi Gerganov（llama.cpp 和 ggml 库的主要作者）

- convert_lora_to_gguf.py
    - 将 LoRA（Low-Rank Adaptation）微调模型转换为 GGUF 格式，使其可以与基础模型结合并在 llama.cpp 中运行。



## 1.3 llama-server

- -m: m是model的缩写。

- --host：指定主机，可以是0.0.0.0（监听所有可用的网络接口，包括localhost）,也可以是localhost，默认为localhost/127.0.0.1。

- --port：指定服务端口，默认端口为8080

- --embedding：指定服务于embedding模型

- --reranking: 指定服务于reranker模型

- --pooling：指定池化类型，可选值none/mean/cls/last/rank。
    
    - 一种将多个token嵌入向量合并成一个固定大小向量的技术。


```shell
llama-server -m bge-reranker-large-f16.gguf --port 11433  --reranking --pooling rank
```


# 2. 实现一个langchain的压缩器

压缩器，用于将检索器检索到的文档列表进行后处理。langchain对压缩器封装的基类为BaseDocumentCompressor，所以压缩器更准确的称呼为“文档压缩器”。

可以参考：CrossEncoderReranker 的实现。

- BaseDocumentCompressor
    - compress_documents(): 抽象方法，需要手动实现。
      1. 传入问题，和文档列表。
      2. 使用requests库，请求llama.cpp启动的reranker服务，得到相似度得分列表。
      3. 根据返回的得分对文档列表进行重排序，将排序后的文档列表返回。


    - acompress_documents(): 非抽象方法，默认即可；也可以自己实现。


In [ ]:
curl http://127.0.0.1:8012/v1/rerank \
    -H "Content-Type: application/json" \
    -d '{
        "model": "some-model",
            "query": "What is panda?",
            "top_n": 3,
            "documents": [
                "hi",
            "it is a bear",
            "The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China."
            ]
    }' | jq

In [12]:
import operator
from typing import Optional, Sequence

from langchain_core.callbacks import Callbacks
from langchain_core.documents import BaseDocumentCompressor, Document

from langchain.retrievers.document_compressors.cross_encoder import BaseCrossEncoder

import requests


class LlamaCppReranker(BaseDocumentCompressor):

    url: str = "http://localhost:8080/reranking"
    
    top_n: int = 3
    """Number of documents to return."""

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Optional[Callbacks] = None,
    ) -> Sequence[Document]:
        payload = {
            # "model": "大小寒学AI",
            "query": query,
            "top_n": self.top_n,
            "documents":[item.page_content for item in documents]
        }
        response = requests.post(self.url, json=payload)
        if response.status_code == 200:
            response_json = response.json()
            print(response_json)
            scores = response_json.get("results", [])
            scores = [score_item.get("relevance_score") for score_item in scores]
            # return []
            docs_with_scores = list(zip(documents, scores))
            # lambda x:x[1]
            result = sorted(docs_with_scores, key=operator.itemgetter(1), reverse=True)
            return [doc for doc, _ in result[: self.top_n]]
        else:
            raise Excpetion("Failed to compress_documents")


In [13]:
import chromadb
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
query = "2024年中东非智能手机的出货量为多少？"

chromadb_client = chromadb.HttpClient(host="localhost", port=8000)
embedding_fn = OllamaEmbeddings(model="nomic-embed-text:latest")
        
vector_store = Chroma(collection_name="rag-video-collection",
                              client=chromadb_client, 
                              embedding_function=embedding_fn
                            )
retriever = vector_store.as_retriever(
            # search_type="mmr",
            # search_kwargs={"k": 16, "fetch_k": 20, "lambda_mult": 0.1}
            search_kwargs={"k": 20}
        )

# Document对象,id/page_content/metadata
docs = retriever.invoke(query)

reranker = LlamaCppReranker(
    url="http://localhost:11433/reranking",
    top_n=4
)
results = reranker.compress_documents(
    query=query,
    documents=docs,
)
# results

{'model': '大小寒学AI', 'object': 'list', 'usage': {'prompt_tokens': 3074, 'total_tokens': 3074}, 'results': [{'index': 0, 'relevance_score': -1.235522747039795}, {'index': 1, 'relevance_score': -3.671241521835327}, {'index': 2, 'relevance_score': -5.785731792449951}, {'index': 3, 'relevance_score': -5.383221626281738}, {'index': 4, 'relevance_score': -1.5460096597671509}, {'index': 5, 'relevance_score': -7.980041027069092}, {'index': 6, 'relevance_score': -6.460325717926025}, {'index': 7, 'relevance_score': -8.532350540161133}, {'index': 8, 'relevance_score': -8.52607536315918}, {'index': 9, 'relevance_score': -6.12175989151001}, {'index': 10, 'relevance_score': -7.699027061462402}, {'index': 11, 'relevance_score': -4.303559303283691}, {'index': 12, 'relevance_score': -7.3094000816345215}, {'index': 13, 'relevance_score': -9.055702209472656}, {'index': 14, 'relevance_score': -7.169929504394531}, {'index': 15, 'relevance_score': -6.98286247253418}, {'index': 16, 'relevance_score': -6.23772

In [14]:
# 1. 添加文档到chromadb当中
# 2. 根据问题进行检索并输出答案

import chromadb
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from sentence_transformers import CrossEncoder

from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

def get_metadata_str(metadata):
    if metadata is None:
        return ""
    _str = ""
    # (key, value)
    for key, value in metadata.items():
        _str += f"{key}:{value}\n"
    return _str

class RagLangChain():
    def __init__(self, 
                 collection_name: str = "rag-video-collection", 
                 host: str = "localhost",
                port: int = 8000):
        self.collection_name = collection_name
        self.host = host
        self.port = port

    def __get_vector_store(self):
        chromadb_client = chromadb.HttpClient(host=self.host, port=self.port)
        embedding_fn = OllamaEmbeddings(model="nomic-embed-text:latest")
        
        vector_store = Chroma(collection_name=self.collection_name,
                              client=chromadb_client, 
                              embedding_function=embedding_fn
                             )
        return vector_store

    def delete_collection(self):
        vector_store = self.__get_vector_store()
        vector_store.delete_collection()
        
    def add_file(self, file_path: str, metadata = None):
        loader = TextLoader(file_path)
        
        docs = loader.load()
        text_spliter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
        all_splits = text_spliter.split_documents(docs)

        if metadata is not None:
            for split_item in all_splits:
                split_metadata = split_item.metadata
                split_item.metadata = {**split_metadata, **metadata}
        
        vector_store = self.__get_vector_store()
        ids = vector_store.add_documents(documents=all_splits)
        return ids

    def __query_vector(self, info):
        vector_store = self.__get_vector_store()
        retriever = vector_store.as_retriever(
            # search_type="mmr",
            # search_kwargs={"k": 16, "fetch_k": 20, "lambda_mult": 0.1}
            search_kwargs={"k": 20}
        )

        # Document对象,id/page_content/metadata
        # docs = retriever.invoke(info["query"])

        # model = CrossEncoder("BAAI/bge-reranker-large")
        # # question = "2024年中东非智能手机的出货量为多少？"
        # scores = model.predict([(info["query"], doc_item.page_content) for doc_item in docs])
        # # [(score, Document), (score, Document), ...]
        # sorted_list = sorted(zip(scores, docs), reverse=True, key=lambda x:x[0])
        # docs = [doc for _, doc in sorted_list]
        # docs = docs[:4]


        # 2. 使用langchain的CrossEncoder进行重排
        # model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-large")
        # compressor = CrossEncoderReranker(
        #     model=model,
        #     top_n=4,
        # )
        compressor = LlamaCppReranker(
            url="http://localhost:11433/reranking",
            top_n=4
        )
        compressor_retriever = ContextualCompressionRetriever(
            base_retriever=retriever,
            base_compressor=compressor,
        )

        docs = compressor_retriever.invoke(info["query"])
        
        # (0, item0), (1, item1)
        for index, doc_item in enumerate(docs):
            print(f"文档片段{index}")
            print(doc_item)
            print("*" * 80)
        # docs_str = "\n\n".join(doc.page_content for doc in docs)
        docs_str = "\n\n".join(f"""
{get_metadata_str(doc.metadata)}
{doc.page_content}""" for doc in docs)
        # print(f"@@@@@{docs_str}")
        return docs_str
        
    def query(self, question: str):
        prompt = ChatPromptTemplate.from_template("""
你是一个问答机器人。你的任务是根据下述给定的已知信息回答用户问题。

已知信息:
{context}
用户问题：
{query}
如果已知信息不包含用户问题的答案，或者已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。
请不要输出已知信息中不包含的信息或答案。
请用中文回答用户问题。
""")
        
        llm = ChatOllama(model="qwen2.5:latest")
        
        output_parser = StrOutputParser()
        # 返回值: {"query": "XXXX", "context": "XXXXX2"}
        chain = ( {"context": self.__query_vector, "query": lambda x: x["query"]} | prompt | llm | output_parser)
        
        result = chain.invoke({"query": question})
        return result


/opt/miniconda3/envs/rag-video/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
rag_chain = RagLangChain()

In [16]:
rag_chain.query(question="2024年中东非智能手机的出货量为多少？")

{'model': '大小寒学AI', 'object': 'list', 'usage': {'prompt_tokens': 3074, 'total_tokens': 3074}, 'results': [{'index': 0, 'relevance_score': -1.235522747039795}, {'index': 1, 'relevance_score': -3.671241521835327}, {'index': 2, 'relevance_score': -5.785731792449951}, {'index': 3, 'relevance_score': -5.383221626281738}, {'index': 4, 'relevance_score': -1.5460096597671509}, {'index': 5, 'relevance_score': -7.980041027069092}, {'index': 6, 'relevance_score': -6.460325717926025}, {'index': 7, 'relevance_score': -8.532350540161133}, {'index': 8, 'relevance_score': -8.52607536315918}, {'index': 9, 'relevance_score': -6.12175989151001}, {'index': 10, 'relevance_score': -7.699027061462402}, {'index': 11, 'relevance_score': -4.303559303283691}, {'index': 12, 'relevance_score': -7.3094000816345215}, {'index': 13, 'relevance_score': -9.055702209472656}, {'index': 14, 'relevance_score': -7.169929504394531}, {'index': 15, 'relevance_score': -6.98286247253418}, {'index': 16, 'relevance_score': -6.23772

'2024年中东非智能手机的出货量为4,199万部。'

# 3. llama.cpp的gguf转换工具

格式转换工具（将其他模型文件转换成gguf格式）

- 安装说明
    - 下载llama.cpp项目到本地
    - 使用conda创建一个虚拟环境，如"gguf-env"
      ```shell
      conda create -n gguf-env python=3.10
      ```
    - 在llama.cpp目录下执行 pip install  -r requirements.txt
    - 【注意】不要单独将这些py工具文件和requirements.txt/requirements目录单独出去进行使用，对最新的gguf-py有依赖。

- convert_hf_to_gguf.py
    -  将 Hugging Face 格式的模型权重（通常是 PyTorch 或 SafeTensors 格式）转换为 GGUF 格式。


```shell
# outtype, choices=["f32", "f16", "bf16", "q8_0", "tq1_0", "tq2_0", "auto"], default="f16"
python convert_hf_to_gguf.py ./models/Qwen2.5-1.5B-Instruct --outtype q8_0 --outfile ./models/Qwen2.5-1.5B-Instruct_q8_0.gguf
```
    
- convert_hf_to_gguf_update.py
    - 用于更新convert_hf_to_gguf.py工具中的get_vocab_base_pre()函数
    - get_vocab_base_pre(): 根据分词器获取模型类型的一个函数。
    - 一般不需要执行这个工具，除非遇到下面的错误：
        - NotImplementedError("BPE pre-tokenizer was not recognized - update get_vocab_base_pre()")
    - 执行这个工具时会遇到的错误：
        - ValueError: Unrecognized model in models/tokenizers/command-r.
        - ValueError: Unrecognized model in models/tokenizers/jais.
        - ValueError: Unrecognized model in models/tokenizers/tekken.
        - ValueError: Unrecognized model in models/tokenizers/exaone.
        - 是因为这些分词器对应的模型，访问需要申请权限。从大陆网段申请/注册的账号为大陆的话，可能会被拒绝。 如果不使用这些模型的话，可以将convert_hf_to_gguf_update.py中models配置中对应的配置注释掉就可以了。

```
 python convert_hf_to_gguf_update.py <huggingface_token>
```

- convert_llama_ggml_to_gguf.py
    - 将旧版的 GGML 格式模型文件转换为新版的 GGUF 格式。
    - GGML文件格式，它是gguf的前身，也是llama.cpp的作者提出。随着llama.cpp项目的演进和发展，这种格式后续会慢慢被替换掉。
    - ggml 是 Georgi Gerganov Machine Learning 的缩写，其命名来源于项目的核心开发者 Georgi Gerganov（llama.cpp 和 ggml 库的主要作者）

- convert_lora_to_gguf.py
    - 将 LoRA（Low-Rank Adaptation）微调模型转换为 GGUF 格式，使其可以与基础模型结合并在 llama.cpp 中运行。



## 3.1 ollama中使用自己的gguf模型

1. Modelfile文件
创建一个Modelfile文件，并写入如下内容.

注意将"./vicuna-33b.Q4_0.gguf"改成自己的gguf文件路径。

```
FROM ./vicuna-33b.Q4_0.gguf
```

2. 创建ollama模型

注意：Modelfile为第1步中创建的Modelfile文件。

命令为：

```shell
# example为模型的名字，可自行修改
ollama create example -f Modelfile
```
3. 在命令行中运行

```
ollama run example
```

4. 推送自己的模型到ollama仓库

![ollama_push](./ollama_push.png)


## 3.2 git lfs 插件

lfs：large file storage,大文件存储。

它是一个git的插件，安装命令为：

1. 安装

[官网地址](https://git-lfs.com/)

    1.1 windows


    从 https://gitforwindows.org/ 下载安装


    1.2 macOS

```shell
brew install git-lfs
```

    1.3 linux

```shell
# apt/deb: 
sudo apt-get install git-lfs
#yum/rpm: 
sudo yum install git-lfs
```

2. 初始化

```shell
# 2. 初始化
git lfs install
```

3. 验证

```shell
# 1. 验证是否安装成功
git lfs --help
# 2. 查看当前git目录下哪些文件是通过git lfs下载的
git lfs ls-files
```
